# Standard Type Library

Pandapower provides a standard type library that allows the comfortable managing of type data for lines and transformers.

The following examples are all based on lines, but can be equally used for the transformer database by specifying element="trafo" instead of element="line" in all standard type functions.

## Basic Pandapower Standard Types

Pandapower comes with generic standard types that you can use to build generic networks:

In [ ]:
from pandapower.create import create_empty_network, create_bus, create_line
from pandapower.std_types import (
    available_std_types,
    create_std_type,
    create_std_types,
    copy_std_types,
    parameter_from_std_type,
    change_std_type,
    find_std_type_by_parameter
)

In [ ]:
line_std_type = "NAYY 4x50 SE"

net = create_empty_network()
available_std_types(net, element="line")

If you create a line with the create_line function, the element parameters are taken from this library:

In [ ]:
b1 = create_bus(net, vn_kv=.4)
b2 = create_bus(net, vn_kv=.4)
lid = create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type=line_std_type, name="test_line")
net.line.loc[lid]

## Create Individual Standard Types

Additionally to the basic pandapower standard types, you can define individual standard types for you pandapower network.

To create a new standard type you need at least the following parameters:

**For Lines**:
- r_ohm_per_km
- x_ohm_per_km
- c_nf_per_km
- max_i_ka

**For Transformers:**
- sn_mva
- vn_hv_kv
- vn_lv_kv
- vk_percent
- vkr_percent
- i0_percent
- pfe_kw
- shift_degree

To define the parameters, create a dictionary with the style {"parameter1": parameter_value1, "parameter2": paramter_value2,...} and save the standard type to your network with the create_std_type function:

In [ ]:
net = create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs"}
create_std_type(net, name="test_type", data=test_type, element="line")
available_std_types(net, element="line")

As you can see the test_type is now available in the standard type library, and you can use it to create a line:

In [ ]:
b1 = create_bus(net, vn_kv=.4)
b2 = create_bus(net, vn_kv=.4)
lid = create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line")
print(net.line.loc[lid])

## Local Standard Type Libraries

Standard tpyes are saved with the network if you use the to_json function and are therefore still available if you load the network again.

If you have a set of standard types that you want to save locally and apply to multiple networks, you can create a set of standard type parameters as a dictionary in the style {"std_type1: {"r_ohm_per_km": r_ohm_per_km,...}, "std_type2": {"r_ohm_per_km": r_ohm_per_km,...},...} and add it to the network with the create_std_types function:

In [ ]:
net = create_empty_network()
linetypes = {"typ1": {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs"},
             "typ2": {"r_ohm_per_km": 0.015, "x_ohm_per_km": 0.01, "c_nf_per_km": 30, "max_i_ka": 0.3, "type": "cs"}}
create_std_types(net, data=linetypes, element="line")
available_std_types(net)

The linetypes dictionary can be easily saved in a local project folder (e.g. using pickle or jsonpickle) and reloaded once it is needed. 

Another possibility is to copy the standard type library of an existing network:

In [ ]:
new_net = create_empty_network()
copy_std_types(from_net=net, to_net=new_net, element="line")
available_std_types(new_net)

## Add Additional Parameters

You can define additional parameters in the standard type library and load them into pandapower. For example, if you want to run a reliability analysis for which you need a hazard rate for each line, you will want to add the hazard rate "h_per_km" for your individual standard type:

In [ ]:
net = create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs", "h_per_km": 0.01}
create_std_type(net, name="test_type", data=test_type, element="line")
available_std_types(net, element="line")

You can see that the parameter "h_per_km" is stored in the standard type database for the new standard type.

We now create two lines, one of the new "test_type" and one with a standard type that does not have the parameter "h_per_km".

In [ ]:
b1 = create_bus(net, vn_kv=.4)
b2 = create_bus(net, vn_kv=.4)
l1 = create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line1")
l2 = create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type=line_std_type, name="test_line2")
net.line

The parameter "h_per_km" ist not in the line table, since it is not a pandapower standard parameter. You can however load the parameter from the database with the parameter_from_std_type function:

In [ ]:
parameter_from_std_type(net, parameter="h_per_km")
net.line

Pandapower adds the parameter to the linetable where it is available. The second line does not have a value for h_per_km, since it is not specified in the standard type database. You can however define a fallback (fill) value for lines that do not have a standard type or that have a standard type for which the desired parameter is not available:

In [ ]:
parameter_from_std_type(net, parameter="h_per_km", fill=0.02)
net.line

## Change Standard Type

The change_std_type function allows changing the standard type of an element.

In [ ]:
net = create_empty_network()
b1 = create_bus(net, vn_kv=.4)
b2 = create_bus(net, vn_kv=.4)
l2 = create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type=line_std_type, name="test_line")
net.line

In [ ]:
change_std_type(net, l2, "NAYY 4x150 SE", element="line")
net.line

## Find Standard Type

It is possible to search for a standard type given a set of parameters:

In [ ]:
find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.1188, "x_ohm_per_km": 0.39})

You can also define a tolerance epsilon to find types with similar parameters:

In [ ]:
find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.12, "x_ohm_per_km": 0.40}, epsilon=.05)